# Web Scraping para Análise de dados

Web Scraping no Portal de vagas de emprego <a href=https://www.indeed.com.br/>Indeed</a>.

## Importando os pacotes

Os pacotes a serem utilizados serão:
- `BeautifulSoup` e `selenium` para *web scraping*
- `numpy` e `pandas` para manipulação de dados

In [1]:
# Importando os pacotes
import bs4
import selenium
import numpy as np
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time

## Web Scraping

Vamos utilizar o `Chromedriver` para o processo de extração. O arquivo após baixado, neste <a href=https://sites.google.com/a/chromium.org/chromedriver/home>link</a>,  deve ser descompactado no mesmo local deste `jupyter notebook`.<br>
Quando criarmos o driver de conexão, será aberta outra janela do nevegador e deve-se manter aberta durante todo o processo de extração.

In [2]:
# criando o driver de conexão
driver = webdriver.Chrome("./chromedriver")

Vamos criar um dataframe para receber os dados de nosso interesse, como *Título*, *Localidade*, *Empresa*, *Salario*, *Tipo de pesquisa* e *Descrição*.

In [3]:
# cria o dataframe para receber os dados
dados = pd.DataFrame(columns=['Titulo', 'Localidade', 'Empresa', 'Classificacao', 'Salario', 'Dias_publicado', 
                              'Tipo_Pesquisa', 'Desc', 'Termo_pesq'])

Vamos realizar a pesquisa no site e pra isso, seguimos foram seguidos os seguintes passos:
1. Acesso ao site: https://www.indeed.com.br
2. Seleção dos filtros de pesquisa de vagas com o termo *Data Science" e localidade "Brasil"
3. Copiamos a url gerada na barra de endereços do navegador

In [4]:
# criando um objeto para o termo
termo_pesquisa = ["data+science", "business+intelligence", "machine+learning", "analista+bi", 
                  "cientista+dados", "analista+dados"]

# criando um objeto para a localidade
local = "brasil"

Então vamos criar a função de busca no site, buscando pelas primeiras 20 páginas de cada termo buscado.

In [5]:
for j in termo_pesquisa:

    for i in range(0,35,1):
        
        driver.get(f"https://www.indeed.com.br/jobs?q={j}&l={local}&start={str(i)}")

        time.sleep(6)
    
        driver.implicitly_wait(14)

        

        # criando um loop para os resultados do scraping
        for vaga in driver.find_elements_by_class_name("result"):

            # coletando o elemento HTML interno
            result_html = vaga.get_attribute('innerHTML')

            # parser do código HTML
            soup = BeautifulSoup(result_html, 'html.parser')

            # Buscamos as tags para análise. 
            # Usaremos blocos try/except para evitar erros na execução, no caso de ua informação não estar disponível na vaga

            # buscando pelo título da vaga
            try:
                title = soup.find("a", class_ = "jobtitle").text.replace('\n', '').strip()
            except:
                title = 'None'

            # buscando pelo localidade
            try:
                location = soup.find(class_ = "location").text
            except:
                location = 'None'

            # buscando pela empresa
            try:
                company = soup.find(class_ = "company").text.replace('\n', '').strip()
            except:
                company = 'None'
            
            # buscando pela Classificação
            try:
                rating = soup.find(class_="ratingNumber").text.replace('\n', '').strip()
            except:
                rating = 'None'

            # buscando pelo salário
            try:
                salary = soup.find(class_ = "salary").text.replace('\n', '').strip()
            except:
                salary = 'None'
                
            # buscando pela quantidade de dias da publicação
            try:
                aging = soup.find(class_="date").text.strip()
            except:
                aging = 'None'

            # buscando pelo tipo de pesquisa (orgânica ou patrocinada)
            try:
                sponsored = soup.find("a", class_ = "sponsoredGray").text
                sponsored = "Sponsored"
            except:
                sponsored = 'Organic'

            # buscando pelo sumário
            sum_div = vaga.find_elements_by_class_name("summary")[0]
            try:
                sum_div.click()
            except:
                close_button = driver.find_elements_by_class_name('popover-x-button-close')[0]
                close_button.click()
                sum_div.click()

            # buscando pelo descrição da vaga
            job_desc = driver.find_element_by_id('vjs-desc').text

            # gravando o resultado em nosso dataframe
            dados = dados.append({"Titulo":title, 
                                  "Localidade":location, 
                                  "Empresa":company, 
                                  "Classificacao": rating, 
                                  "Salario":salary, 
                                  "Dias_publicado": aging, 
                                  "Tipo_Pesquisa":sponsored, 
                                  "Desc":job_desc, 
                                  "Termo_pesq":j},
                                 ignore_index = True)

Vamos olhar a dimensão do dataframe.

In [6]:
# exibindo a quantidade de linhas e colunas
print(f'O dataframe possui:\n{dados.shape[0]} linhas e {dados.shape[1]} colunas')

O dataframe possui:
3150 linhas e 9 colunas


Legal! Agora temos dados para trabalhar, mas antes vamos salvar uma cópia em disco.

In [7]:
# salvando os dados em disco
dados.to_csv('resultado/dados.csv', encoding='utf-8')